# Ray start/stop

In [ ]:
ray.init(ignore_reinit_error=True)

In [ ]:
ray.shutdown()

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_rows', 99)
pd.set_option('display.max_columns', 99)
import pandas_bokeh
pandas_bokeh.output_file("data/bokeh_output.html")

import polygon_df as pdf
import polygon_ds as pds
import polygon_s3 as ps3
import bar_samples as bs
import bar_labels as bl
import bar_workflow as bw
import utils_filters as ft

import matplotlib as mpl
from matplotlib import style
style.use('fivethirtyeight')
mpl.rcParams['figure.figsize'] = [20, 7]

# df.style.background_gradient(cmap='coolwarm', axis=None, vmin=-1, vmax=1)  # 'Blues', 'coolwarm', 'RdBu_r' & 'BrBG'
# pd.set_option('precision', 2)
# .plot_bokeh(sizing_mode="scale_height")

# Sample tick dates


In [40]:
# set sampling params
symbol = 'VTI'
start_date = '2020-11-01'
end_date = '2020-11-05'
date = '2020-11-30'

thresh = {
    # max thresholds
    'renko_return': 'jma_return',
    'renko_reveral_multiple': 2,
    'renko_range_frac': 20,
    'max_duration_sec': 60*60*0.5,  # 1/2 hour
    # min thresholds
    'min_duration_sec': 10,
    'min_tick_count': 30,
    # label params
    'label_reward_ratios': list(np.arange(2.5, 11, 0.5)),
}

ray_on = True

if ray_on:
    import ray
    ray.init(ignore_reinit_error=True)


2021-01-01 20:27:04,463	INFO worker.py:666 -- Calling ray.init() again after it has already been called.


In [42]:

bar_dates = bw.bar_dates_workflow(symbol, start_date, end_date, thresh, ray_on)


(pid=17870) VTI 2020-11-04 trying to get data from local file...
(pid=17871) VTI 2020-11-02 trying to get data from local file...
(pid=17868) VTI 2020-11-03 trying to get data from local file...
(pid=17869) VTI 2020-11-05 trying to get data from local file...


In [45]:
tdf = bar_dates[0]['ticks_df']

In [54]:
tdf.status.value_counts()

clean                  49783
after_hours              670
irregular_condition      293
timestamps_delta         122
outlier_filter            12
Name: status, dtype: int64

In [53]:
len(tdf[tdf.status=='clean'])/tdf.shape[0]

0.978439465408805

In [ ]:
stacked_df = bw.fill_gaps_dates(bar_dates, fill_col='jma_close')

stats_df = bw.bar_dates_stats(stacked_df)

daily_stats_df = pd.merge(left=daily_stats_df, right=stats_df, left_on='date', right_on='date')

In [ ]:
stacked_df, daily_stats_df.set_index('date_time')